Funcion para convertir el dataset .tfrecord en formato para trabajar con yolo

In [ ]:
import os
import tensorflow as tf
import cv2
import numpy as np

datasets = {
    'train': 'dataset_folder/train/Stades.tfrecord',
    'test': 'dataset_folder/test/Stades.tfrecord',
    'valid': 'dataset_folder/valid/Stades.tfrecord'
}

output_dir = "dataset_detection"
image_dir = os.path.join(output_dir, "images")
label_dir = os.path.join(output_dir, "labels")

for split in ['train', 'valid', 'test']:
    os.makedirs(os.path.join(image_dir, split), exist_ok=True)
    os.makedirs(os.path.join(label_dir, split), exist_ok=True)

# Función para decodificar el TFRecord
def _parse_function(proto):
    keys_to_features = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64)  
    }
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)

    image = tf.io.decode_jpeg(parsed_features['image/encoded'], channels=3)
    width = tf.cast(parsed_features['image/width'], tf.int32)
    height = tf.cast(parsed_features['image/height'], tf.int32)

    label = tf.sparse.to_dense(parsed_features['image/object/class/label'], default_value=-1)

    label = tf.cond(tf.size(label) > 0, lambda: label[0], lambda: tf.cast(-1, tf.int64))

    return image, width, height, label

# Función para convertir TFRecord a YOLO
def convert_tfrecord_to_yolo(tfrecord_path, split):
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    dataset = dataset.map(_parse_function)

    i = 0
    for image, width, height, label in dataset:
        label_value = label.numpy()  

        if label_value == 0:
            continue  

        image_np = image.numpy()

        image_filename = f"{split}_{i}.jpg"
        image_path = os.path.join(image_dir, split, image_filename)
        cv2.imwrite(image_path, cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))

        label_filename = f"{split}_{i}.txt"
        label_path = os.path.join(label_dir, split, label_filename)

        with open(label_path, "w") as f:
            x_center, y_center, width_norm, height_norm = 0.5, 0.5, 1.0, 1.0
            f.write(f"0 {x_center} {y_center} {width_norm} {height_norm}\n")

        i += 1  

for split, tfrecord_path in datasets.items():
    convert_tfrecord_to_yolo(tfrecord_path, split)

dataset_yaml = f"""train: {image_dir}/train/
val: {image_dir}/valid/
test: {image_dir}/test/

nc: 1
names: ['stadium']
"""

with open(os.path.join(output_dir, "dataset.yaml"), "w") as f:
    f.write(dataset_yaml)



Implementacion del modelo de yolov8 para entrenarlo sobre nuestro dataset

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt

model = YOLO("yolov8n.pt")

        
def train_model_with_checkpoints():
    results = model.train(
        data="dataset.yaml",  
            epochs=7,
                imgsz=640,
                save_period=1,  
                project='runs/train',  
                name='exp',  
            )

    return results

train_results = train_model_with_checkpoints()

last_checkpoint = 'runs/train/exp/weights/best.pt'
model = YOLO(last_checkpoint)

evaluation_results = model.val()
